# Projeto Final de Ciência dos Dados. ( PkmnID)

## Algoritmo de predição da categoria de Pokémons por meio de suas imagens.
### O algoritmo realiza a extração e a clusterização de features de imagens por meio do método \"Bag of Visual Words\" (BOVW),classifica-as utilizando o método de machine learning \"Random Forest\" e prevê a categoria de Pokémons por meio de novas imagens.

In [ ]:
!pip install opencv-contrib-python
import cv2
import os
import os.path
import numpy as np
import math
import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(0)

TRAIN_DIR = 'Assets//Data_Train'
TEST_DIR = 'Assets//Data_Test'

NUM_CLUSTERS = 40

In [ ]:
## Máquina de controle:

# Extrai as features de todas as imagens novamente.
RE_EXTRACT_FEATURES = 0

# Re-extrai os nomes das imagens (com os caminhos adequados), de cada pasta.
RE_ACQUIRE_IMG_NAMES = 0

# Re-constrói o dicionário de features que permite a análise exploratória.
RE_CREATE_FEATURE_DICT = 0

# Cria arquivo Json para dicionário de Features.
CREATE_JSON = 0

# Constrói dataframe com dados do dicionário de features.
CREATE_FEATURE_DATAFRAME = 0

# Mostra em quais 'n' pokémons, cada feature é mais proeminente.
SHOW_TOP_N_FOR_FEATURES = 0

# Re-treina os models.
FIT_MODELS = 0

# Produz matrizes de confusão para todos os modelos.
PLOT_CONFUSION_MATRIXES = 0

# Produz uma lista com a métrica precision@n para todos os modelos.
PRECISION_AT_N = 0


## 1- Extração de features de imagens: Bag of Visual Words
### Uma vez que o dataset se trata de um conjunto de imagens de diferentes Pokémons, é necessário inicialmente extrair features dessas imagens, através do método "Bag of Visual Words".
### Com as imagens transformadas em features clusterizadas, elas são separadas em categorias de treino e teste, que serão utilizadas posteriormente pelo algoritmo de machine learning.
### O código abaixo realiza essas duas etapas:
#### Obs: Código produzido com a assistência do Prof. Fábio Ayres

In [ ]:
def get_img_names(TRAIN_DIR = TRAIN_DIR, TEST_DIR = TEST_DIR):

    TRAIN_IMG = []
    TEST_IMG = []
    TRAIN_LABEL = []
    TEST_LABEL = []

    for train, test in zip(os.listdir(TRAIN_DIR), os.listdir(TEST_DIR)): #Tecnicamente são iguais, mas não custa garantir.
        for img_train, img_test in zip(os.listdir(os.path.join(TRAIN_DIR,train)), os.listdir(os.path.join(TEST_DIR,test))):
            TRAIN_IMG.append(os.path.join(TRAIN_DIR,train,img_train))
            TEST_IMG.append(os.path.join(TEST_DIR,test,img_test))
            TRAIN_LABEL.append(train)
            TEST_LABEL.append(test)

    return TRAIN_IMG, TEST_IMG, TRAIN_LABEL, TEST_LABEL

def cria_vocabulario(imagens, num_clusters):
    km = cv2.BOWKMeansTrainer(num_clusters)
    akaze = cv2.KAZE_create()
    for p in imagens:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        mask = np.ones(img.shape)
        kp, desc = akaze.detectAndCompute(img, mask)
        km.add(desc)
    return km.cluster()

def representa(vocab, img):
    kaze = cv2.KAZE_create()
    kp = kaze.detect(img)
    bowdesc = cv2.BOWImgDescriptorExtractor(kaze, cv2.FlannBasedMatcher())
    bowdesc.setVocabulary(vocab)
    return bowdesc.compute(img, kp)

def transforma_imagens(imagens, vocab):
    X = []
    for p in imagens:
        img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
        X.append(representa(vocab, img).flatten())
    return np.array(X)

def show_example(path = os.listdir("Testes/Testes/")[0], Plot = True):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, dsize=(120, 120))
    if Plot:
        plt.imshow(img_resized, cmap='gray', vmin=0, vmax=255)
    return representa(vocab, img_resized)

if RE_ACQUIRE_IMG_NAMES:
    TRAIN_IMG, TEST_IMG, TRAIN_LABEL, TEST_LABEL = get_img_names()

if RE_EXTRACT_FEATURES:
    vocab = cria_vocabulario(TRAIN_IMG, NUM_CLUSTERS)
    X_train = transforma_imagens(TRAIN_IMG, vocab)
    X_test = transforma_imagens(TEST_IMG, vocab)
    y_train = TRAIN_LABEL
    y_test = TEST_LABEL

## 2 - Análise Exploratória:
### Para realizar a análise exploratória seguiremos alguns passos:
### 2.1 - Extrair histograma:
### O código abaixo cria o histograma de frequências relativas das features de todas as imagens do dataset escolhido.

In [ ]:
if RE_CREATE_FEATURE_DICT:
        origin_dir = 'Assets/Data_Train'
        Hist_Dict = {}
        for pkmn in os.listdir(origin_dir):
                Hist_Dict[pkmn] = []
                current_dir = os.path.join(origin_dir,pkmn)
                for k, img in enumerate(os.listdir(current_dir)):
                        Hist_Dict[pkmn].append(show_example(os.path.join(current_dir,img), Plot = False))

        if CREATE_JSON:
                with open('feature_dict.txt', 'w') as filename:
                        a = json.dumps(str(Hist_Dict))
                        filename.write(a)
else:
        with open("feature_dict.txt", 'r') as filename:
                Hist_Dict = json.load(filename)

## 2.2 - Criar um dataframe para trabalhar melhor com o dataset:

In [ ]:
if CREATE_FEATURE_DATAFRAME:    
    lista = []
    lista_nomes = os.listdir('Assets/Data_Train')
    for k in Hist_Dict:
        x = pd.Series(Hist_Dict[k]).mean()
        x = pd.Series(x[0])
        lista.append(x)
    df_medias = pd.DataFrame(lista, index = lista_nomes)
else:
    df_medias = pd.DataFrame([[1],[1]])

## Tabela das frequências relativas médias de cada feature por pokémon:

In [ ]:
df_medias.head()

## 2.3 - Calculando os valores médios dos dados:
### Nesta etapa foi calculado os valores médios dos dados, e em sequência foram aproximados do ponto (0,0), origem do sistema.

In [ ]:
df_medias = df_medias - (1/40)

In [ ]:
df_medias.sum(axis=1)

In [ ]:
normas = (df_medias*df_medias).sum(axis=1)
for m in normas.index:
    df_medias.loc[m] = df_medias.loc[m]/np.sqrt(normas[m])

## 2.4 - Comparação entre os Pokémons:
### Com base nos valores calculados anteriormente, foi criada a tabela seguinte, que mostra o quanto os Pokémons são semelhantes entre si, sendo 1 a semelhança máxima, e -1 o oposto.

In [ ]:
df_compara = df_medias.dot(df_medias.transpose())
df_compara

### Podemos observar que alguns Pokémons possuem muitas semelhanças pois apresentam as mesmas features em abundância (na média)."

In [ ]:
if SHOW_TOP_N_FOR_FEATURES:    
    monstros = []
    for feat in range(NUM_CLUSTERS):
        monstros.append(sorted(df_medias.nlargest(n=5, columns=[feat]).index) + [feat])
    x = sorted(monstros)
    pprint(x)

### A soma das colunas da tabela anterior mostra quais Pokémons são mais difíceis de distinguir.

In [ ]:
df_compara.sum(axis = 1).sort_values(ascending = False)

## 3 - "Machine Learning" e Classificação:
### O método de aprendizado de máquina e classificação utilizado foi o "Random Forest Classifier", assim como "Logistic Regression Classifier" e "KNearesNeighbors Classifier".

In [ ]:
if FIT_MODELS:
    # Random forest
    randf = RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators = 100)
    randf.fit(X_train, y_train)

    # KNearestNeighbors
    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(X_train, y_train)

    # Logistic regression
    logit = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
    logit.fit(X_train, y_train);

    # Método Nearest Centroid, não utilizado.
    # from sklearn.neighbors.nearest_centroid import NearestCentroid
    # clf4 = NearestCentroid()
    # clf4.fit(X_train, y_train)

    # Método Support Vector Machine, não utilizado.
    # from sklearn import svm
    # clf5 = svm.SVC(gamma='scale')
    # clf5.fit(X_train, y_train)

### Abaixo armazenamos os modelos numa estrutura que nos será mais acessível.

In [ ]:
if FIT_MODELS:
    models = {'Random Forest': randf,
            'KNearestNeighbors': neigh,
            'Losgistic Regression': logit}
else:
    models = {0:0}

## 3.1 - Análise das classificações realizadas pelo modelo:
### A matriz de confusão abaixo mostra em mais detalhes os erros e acertos do classificador. É possível identificar que na maioria das vezes que o modelo falhou, ele identificou erroneamente o Pokémon como sendo uma "Jigglypuff" ou um "Arcanine".

#### Obs: A função *plot_confusion_matrix* abaixo não é de nossa autoria, e sua versão original pode ser encontrada no seguinte endereço: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=True,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    fig, ax = plt.subplots(figsize = (16,16))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

def multi_confusion_mtx(X_test, y_test, model_dict):
    for k,v in model_dict.items():
        plot_confusion_matrix(y_test, v.predict(X_test), classes=v.classes_,
                            title='Normalized confusion matrix for model %s' % k)
        plt.show()

def precision_at_n(model):
    hits, miss = 0, 0
    for img, label in zip(TEST_IMG, TEST_LABEL):
        rep = representa(vocab, cv2.imread(img))
        top3 = pd.Series(model.predict_proba(rep)[0], index = os.listdir('Assets/Data_Test')).nlargest(3)
        if label in top3.index.tolist():
            hits += 1
        else:
            miss += 1
            
    return hits, miss, hits/(hits+miss)

def show_guess(path, model):
    return pd.Series(model.predict_proba(show_example(path))[0], index = model.classes_).sort_values(ascending = False)

np.set_printoptions(precision=2)

In [ ]:
if PLOT_CONFUSION_MATRIXES and FIT_MODELS:
    multi_confusion_mtx(X_test, y_test, models)

In [ ]:
if PRECISION_AT_N and FIT_MODELS:
    pprint({model_name: precision_at_n(model) for (model_name, model) in models.items()})